In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance

In [2]:
df = pd.read_csv('./data/df_features.gz')

In [3]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Diff', 'Review_Month',
       'Review_Year', 'Country', 'City', 'Pet', 'Purpose', 'Whom', 'Room',
       'Length', 'Device', 'Room_Recode', 'Nationality_Recode',
       'Length_Recode', 'Close_Landmarks'],
      dtype='object')

### Landmarks

In [4]:
lm = pd.read_excel('./data/landmarks.xlsx')
lm

,City,latitude,longitude
0,Amsterdam,52.366539,4.887895
1,Barcelona,41.387318,2.166822
2,London,51.511056,-0.126601
3,Milan,45.468481,9.187097
4,Paris,48.862269,2.332588
5,Vienna,48.200836,16.365660


In [5]:
df_compact = df[['Hotel_Address','lat','lng','City']].groupby(['Hotel_Address']).min().reset_index()

In [6]:
result = pd.merge(df_compact, lm, on='City')
result = result.dropna()

In [7]:
result['Dist_Center'] = result.apply(lambda x: distance((x[1],x[2]), (x[4],x[5])).m, axis=1)
result = result[['Hotel_Address','Dist_Center']]
result.head()

,Hotel_Address,Dist_Center
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,2024.205617
1,2e Hugo de Grootstraat 2 6 Westerpark 1052 LC ...,1255.826260
2,Albert Cuypstraat 2 6 Oud Zuid 1072 CT Amsterd...,1345.380072
3,Aletta Jacobslaan 7 Slotervaart 1066 BP Amster...,4458.063088
4,Amstelstraat 17 Amsterdam City Center 1017 DA ...,727.887377


In [8]:
df_updated = pd.merge(df, result, on='Hotel_Address', how='left')

In [9]:
df_updated.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Diff', 'Review_Month',
       'Review_Year', 'Country', 'City', 'Pet', 'Purpose', 'Whom', 'Room',
       'Length', 'Device', 'Room_Recode', 'Nationality_Recode',
       'Length_Recode', 'Close_Landmarks', 'Dist_Center'],
      dtype='object')

#### Save Results

In [10]:
df_updated.to_csv("./data/df_features.gz", index_label=False, compression="gzip")